In [ ]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta, date
from sqlalchemy import create_engine,create_engine, MetaData, Table, select, and_, func, not_
from sqlalchemy.dialects.postgresql import JSON
from sqlalchemy.sql.expression import cast

In [ ]:
# noms des vagues exclues du tirage
excluded_last_nps = ['U1','U2','U3','U4','U4b','U5','U6'] 

# nom de la nouvelle vague NPS (dernière vague de excluded_last_nps + 1)
new_nps='U7'

In [ ]:
# on importe le csv
contacts = pd.read_csv('~/contacts.csv',sep=',')

# on garde les colonnes qui nous intéresse
contacts = contacts[['email','département','nps','unsubscribed.1']]

# on indique si le département est renseigné et est éligible 
eligible_departements = ['08','22','25','29','34','35','56','58','67','71','84','93','94','973'] # départements éligibles
contacts['isEligibleDept'] = contacts.département.str.startswith(tuple(eligible_departements))

# on retraite les utilisateurs qui n'ont pas de département valide
contacts = contacts.loc[contacts.isEligibleDept==True]

# on retraite les désabonnés
contacts = contacts.loc[contacts['unsubscribed.1']=='f']

# on retraite les vagues exclues du tirage
contacts['excluded'] = contacts.nps.isin(excluded_last_nps)
selected_contacts = contacts.loc[(contacts.excluded==False)]

# on créé un dataframe contenant seulement EMAIL et NPS
contacts = contacts[['email','nps']]

In [ ]:
# on vérifie que les vagues exclues du tirage ont bien été exclues. Output attendu : 
# [Series([], Name: nps, dtype: int64)]
selected_contacts.nps.value_counts()

In [ ]:
# on tire 2500 lignes aléatoirement
selected_contacts = selected_contacts.sample(n=2500)

# nom de la nouvelle vague NPS (dernière vague de excluded_last_nps + 1)
selected_contacts['nps']=new_nps

In [ ]:
# on regarde si le tirage c'est bien passé. Output attendu : 
# [email = 2500, nps = 1 (new_nps), excluded = 1 (False)]
selected_contacts.nunique()

In [ ]:
# on exporte la nouvelle vague de contact NPS
selected_contacts.to_csv('nps_'+new_nps+'.csv')